In [1]:
from collections import defaultdict
import csv
from datetime import datetime
import os
import re

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pytz import timezone
from sklearn.externals import joblib
import torch
from torch import nn
from tqdm import tqdm_notebook

from load_dataset import AudioDataset
from train_coarse import VGG_alt, device  #  TEST_LOADER
# from train_branches import VGG_alt as VGG_alt_fine, get_label_range

##  edit the optimizer  'rmsprop' or 'adam' for different model test 
from train_branches_20190711_1_one_part_one_model import VGG_alt as VGG_alt_fine, get_label_range

# from train_branches_20190711_2_one_part_one_model_no_dropout import VGG_alt as VGG_alt_fine, get_label_range

/data/anaconda/envs/dcase/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Config

In [2]:
DATE = datetime.now(timezone('US/Pacific')).strftime('%Y%m%d_%H%M%S')
print(f'DATE: {DATE}')

DATE: 20190715_141637


In [70]:
LINUX = True

#FINE = False
#COARSE = True

COARSE_WITH_FINE_TOP_LEVEL_MODEL = True

FINE_TO_COARSE_LOGIC = False

VGG_EMBED = True
L3_EMBED = False

if not LINUX:
    basepath = r"D:\repos\Data-Processing\audio\test"
else:
    basepath = '/dcase/spec_vgg/validate'                                  # Validate / compute metrics
    #basepath = '/dcase/datasets-dcase-2019-5/audio-eval/features/spec_vgg'  # Final Eval set
NUM_COARSE_LABELS = 8
NUM_TOTAL_LABELS = 37
NUM_FINE_LABELS = NUM_TOTAL_LABELS - NUM_COARSE_LABELS

COARSE_PREDICT_THRESHOLD = 0.5  # what else?

RESULT_CSV_COARSE_PATH = f'csvs/results-{DATE}-coarse.csv'
RESULT_CSV_FINE_PATH_TEMPLATE = f'csvs/results-{DATE}-fine-%d.csv' 
RESULT_CSV_FINAL_PATH = f'csvs/results-{DATE}-final.csv'

total_files = 0

4125

In [25]:
if not LINUX:
    COARSE_CHECKPOINT = "models/20190605_085047_best_epoch_1_val_loss=0.2655.ckpt"
    FINE_CHECKPOINT_BASE = 'models'
else:
    #COARSE_CHECKPOINT = '/dcase/trained-models-branches/20190607_085504_0.773.ckpt'
    #COARSE_CHECKPOINT =  '/dcase/trained-models-branches/20190610_083507_coarse=0.777_fine=0.644.ckpt'
    #COARSE_CHECKPOINT = '/dcase/models/20190610_083507_coarse=0.777_fine=0.644.ckpt'
    
    #Best Coarse from Daniel:
    COARSE_CHECKPOINT = '/dcase/models/20190610_170308_coarse=0.787_fine=0.646.ckpt'
    
    # Best Fine form Daniel:
#     COARSE_CHECKPOINT = '/dcase/models/20190609_230306_coarse=0.769_fine=0.656.ckpt'
    
#    FINE_CHECKPOINT_BASE = '/dcase/output/models_Jianyu/one-part-1-models-logmel-best7-rmsprop-learningratechangeMuchbiggerinCallback'

#     FINE_CHECKPOINT_BASE = '/dcase/output/models_Jianyu/one-part-1-models-logmel-best6-originalmodel_rmsprop-learningratechangebiggerinCallback'
#     FINE_CHECKPOINT_BASE = '/dcase/output/models_Jianyu/one-part-1-models-logmel-best5-rmsprop-learningratechangebiggerinCallback'

    FINE_CHECKPOINT_BASE = '/dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmsprop-learningratechangebiggerinCallback'
#     FINE_CHECKPOINT_BASE = '/dcase/output/models_Jianyu/one-part-1-models-logmel-best3-rmsprop-dropoutlow'
#     FINE_CHECKPOINT_BASE = '/dcase/output/models_Jianyu/one-part-1-models-logmel-best2-rmsp'      ## best result so far
#     FINE_CHECKPOINT_BASE = '/dcase/output/models_Jianyu/one-part-1-models-logmel-best'
#     FINE_CHECKPOINT_BASE = '/dcase/trained-models-branches'
#     FINE_CHECKPOINT_BASE = '/dcase/output/models/best'

FINE_CKPT_REGEX = r'.+_coarse=(\d+)_best_.*\.ckpt'

## Find checkpoints for fine categories

Assumes there is at most 1 ckpt file for each coarse category

In [26]:
ckpt_paths_fine = defaultdict(str)
for f in os.listdir(FINE_CHECKPOINT_BASE):
    m = re.match(FINE_CKPT_REGEX, f)
    if m:
        coarse_idx = int(m.group(1))
        ckpt_paths_fine[coarse_idx] = os.path.join(FINE_CHECKPOINT_BASE, f)
ckpt_paths_fine

defaultdict(str,
            {5: '/dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmsprop-learningratechangebiggerinCallback/20190712_205505_coarse=5_best_epoch_3_val_loss=0.2760.ckpt',
             4: '/dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmsprop-learningratechangebiggerinCallback/20190712_205505_coarse=4_best_epoch_9_val_loss=0.2413.ckpt',
             0: '/dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmsprop-learningratechangebiggerinCallback/20190712_205505_coarse=0_best_epoch_5_val_loss=0.2760.ckpt',
             3: '/dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmsprop-learningratechangebiggerinCallback/20190712_205505_coarse=3_best_epoch_17_val_loss=0.1979.ckpt',
             6: '/dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmsprop-learningratechangebiggerinCallback/20190712_205505_coarse=6_best_epoch_11_val_loss=0.1304.ckpt',
             1: '/dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmspr

## Set up label names

In [27]:
# re-orders output as [fine, coarse] to match demo .csv file (maybe not necessary)
def reorder_labels_for_submission(list_or_array):
    if type(list_or_array) == torch.Tensor:
        list_or_array = list(list_or_array.to('cpu').numpy())
    list_or_array = list(list_or_array)
    return list_or_array[8:] + list_or_array[0:8]

In [28]:
# process label names
label_names = joblib.load('label_order.pkl')
label_names = [re.sub('_presence', '', label_names[i]) for i in range(len(label_names))]

In [29]:
#label_names
# total_files = 0

In [66]:
def make_results_csv(csv_out_path_template, model_class, ckpt_path, use_fine_model=False, coarse_idx=None,
                    label_start=None, label_end=None, num_fine_classes=None):
    # Skip if path to checkpoint doesn't exit (happens for single-class fine models that don't exist)
    if not ckpt_path:
        return
    
    if use_fine_model:
        csv_out_path = csv_out_path_template % coarse_idx
        model = model_class(num_fine_classes)
    else:
        # Coarse (high-level) model.
        csv_out_path = csv_out_path_template 
        model = model_class()

    # Initialize model weights from checkpoint.
    ckpt = torch.load(ckpt_path)
    model.load_state_dict(ckpt)
    model.eval()    
        
    print(f'Making results file: {csv_out_path}')
    with open(csv_out_path, 'w') as c:
        writer = csv.writer(c, delimiter=',')

        header = ['audio_filename'] + reorder_labels_for_submission(label_names)
        writer.writerow(header)
        data_rows = []
        for filename in tqdm_notebook(os.listdir(basepath)):
            # Only eval on non-augmented validation set data. TODO: remove extraneous files from validation dataset
            if '_pitch_changes-0_volume_changes-0_background_changes-None-None' not in filename:
                continue
            audio_filename = filename[0:9] + '.wav'
#             total_files = total_files + 1
            
            ###### load data for validation 
            ###### 
            spectrogram, vgg, label = joblib.load(os.path.join(basepath, filename))
    #         print(spectrogram.shape, emb.shape, label.shape)
            spectrogram = np.expand_dims(spectrogram, axis=0)
            spectrogram = np.expand_dims(spectrogram, axis=0)

    #         print(vgg.flatten().shape)
            vgg = torch.from_numpy(vgg.flatten().reshape((1, 1280)))
            spectrogram = spectrogram.astype(np.float32)
            spectrogram = torch.from_numpy(spectrogram)
    #         print(spectrogram.shape, emb.shape, label.shape)
            in_data = (spectrogram, vgg)

            with torch.no_grad():
                results = model(in_data)
#                 print("results length1")  # 37 
#                 print(len(results[0]))    # 37 
                results = torch.sigmoid(results[0])
#                 print("results length2")  # 37 
#                 print(len(results))       # 37 
                #print(label)
                #print(results)
                #print()
                label_true = np.where(label == 1)[0]
                label_name = [label_names[i] for i in label_true]
    #             print(label_name)
    #             img_array = img_array.reshape((16, 16))
    #             plt.imshow(img_array, cmap='hot', interpolation='nearest')
    #             plt.show()
    #             print(results.shape)

                results = results.detach().numpy()
                if use_fine_model:
                    print("use_fine_model")
                    if FINE_TO_COARSE_LOGIC:
                        print("FINE_TO_COARSE_LOGIC is true")
                        coarse_labels = []
                        fine_label_names = label_names[NUM_COARSE_LABELS:]
                        coarse_label_dict = {i: [] for i in range(NUM_COARSE_LABELS)}
                        for i, r in enumerate(results):
        #                     print(fine_label_names[i][0])
                            coarse_label_dict[int(fine_label_names[i][0])-1].append(r)
                        for i in range(NUM_COARSE_LABELS):
                            coarse_labels.append(max(coarse_label_dict[i]))
        #                 print('coarse labels', coarse_labels)
                        results = coarse_labels + list(results)
                    else:
                        print("FINE_TO_COARSE_LOGIC is false")
                        full_results = np.zeros(NUM_TOTAL_LABELS)
                        full_results[label_start : label_end] = results
                        results = full_results
                        
                elif COARSE_WITH_FINE_TOP_LEVEL_MODEL:
                    results = list(results)
                else:
                    # Coarse model.
                    results = list(results) + [0 for i in range(NUM_FINE_LABELS)]

                results = reorder_labels_for_submission(results)
                #print (results)
                data_rows.append([audio_filename] + results)
    #             print([audio_filename] + results)
        writer.writerows(data_rows)
    
    # Clean up.
    del ckpt
    del model
    torch.cuda.empty_cache()

In [31]:
#label_names
from sklearn.externals import joblib
Test_data = joblib.load('/dcase/spec_vgg/label_to_files_test.zip')
# test[1]

# print(type(Test_data[0]))
# print(len(Test_data[0]))

In [32]:
annotations = ['1_engine', '2_machinery-impact', '3_non-machinery-impact', '4_powered-saw', '5_alert-signal', '6_music', '7_human-voice', '8_dog']


In [75]:
def make_fine_results_csv(csv_out_path_template, model_class, ckpt_path, use_fine_model=False, coarse_idx=None,
                    label_start=None, label_end=None, num_fine_classes=None):
    # Skip if path to checkpoint doesn't exit (happens for single-class fine models that don't exist)
    if not ckpt_path:
        return
    
    if use_fine_model:
        csv_out_path = csv_out_path_template % coarse_idx
        model = model_class(num_fine_classes)
    else:
        # Coarse (high-level) model.
        csv_out_path = csv_out_path_template 
        model = model_class()

    # Initialize model weights from checkpoint.
    ckpt = torch.load(ckpt_path)
    model.load_state_dict(ckpt)
    model.eval()
    target_annotation = annotations[coarse_idx]
    print(coarse_idx, "     ", target_annotation)
        
#     print(f'Making results file: {csv_out_path}')
    with open(csv_out_path, 'w') as c:
        writer = csv.writer(c, delimiter=',')

        header = ['audio_filename'] + reorder_labels_for_submission(label_names)
        writer.writerow(header)
        data_rows = []
        for filename in tqdm_notebook(os.listdir(basepath)):
#         for index in range (0, len(Test_data[coarse_idx])):
#             test_file = Test_data[coarse_idx][index]
            # Only eval on non-augmented validation set data. TODO: remove extraneous files from validation dataset
#             print(test_file)
#             if '_pitch_changes-0_volume_changes-0_background_changes-None-None' not in test_file:
            if '_pitch_changes-0_volume_changes-0_background_changes-None-None' not in filename:
                continue
            audio_filename = filename[0:9] + '.wav'
            global total_files
            total_files = total_files  + 1
#  
#                        print(test_file)
#             audio_filename = test_file[0:9] + '.wav'
            ###### load data for validation 
            ###### 
            spectrogram, vgg, label = joblib.load(os.path.join(basepath, filename))
            
            
            
            ### maybe not need to add this because when evaluating a certain class, 
            ### the program only evaluates what is clip annotated with that class
#             label_true = np.where(label == 1)[0]
#             label_name = [label_names[i] for i in label_true]
# #             print(label_name)
#             right_file = False
#             for index in range(0, len(label_name)):
#                 if(target_annotation == label_name[index]):
#                     right_file = True
                
#             if(right_file == False):
# #                 print(label_name, " don't include for test")
#                 continue
    
                
#             spectrogram, vgg, label = joblib.load(os.path.join(basepath, test_file))
#             print('spectrogram.shape, label.shape')
#             print(spectrogram.shape, emb.shape, label.shape)
            spectrogram = np.expand_dims(spectrogram, axis=0)
            spectrogram = np.expand_dims(spectrogram, axis=0)
#             print('spectrogram.shape, label.shape')
#             print(spectrogram.shape,label.shape)

    #         print(vgg.flatten().shape)
            vgg = torch.from_numpy(vgg.flatten().reshape((1, 1280)))
            spectrogram = spectrogram.astype(np.float32)
            spectrogram = torch.from_numpy(spectrogram)
    #         print(spectrogram.shape, emb.shape, label.shape)
            in_data = (spectrogram, vgg)

            with torch.no_grad():
                results = model(in_data)
                results = torch.sigmoid(results[0])
                label_true = np.where(label == 1)[0]
                label_name = [label_names[i] for i in label_true]
#                 print(label_name)

                results = results.detach().numpy()
                if use_fine_model:
#                     print("use_fine_model")
                    if FINE_TO_COARSE_LOGIC:
                        print("FINE_TO_COARSE_LOGIC is true")
                        coarse_labels = []
                        fine_label_names = label_names[NUM_COARSE_LABELS:]
                        print(fine_label_names)
                        coarse_label_dict = {i: [] for i in range(NUM_COARSE_LABELS)}
                        for i, r in enumerate(results):
        #                     print(fine_label_names[i][0])
                            coarse_label_dict[int(fine_label_names[i][0])-1].append(r)
                        for i in range(NUM_COARSE_LABELS):
                            coarse_labels.append(max(coarse_label_dict[i]))
        #                 print('coarse labels', coarse_labels)
                        results = coarse_labels + list(results)
                    else:
#                         print("FINE_TO_COARSE_LOGIC is false")
                        full_results = np.zeros(NUM_TOTAL_LABELS)
                        full_results[label_start : label_end] = results
                        results = full_results
                        
                elif COARSE_WITH_FINE_TOP_LEVEL_MODEL:
                    results = list(results)
                else:
                    # Coarse model.
                    results = list(results) + [0 for i in range(NUM_FINE_LABELS)]

                results = reorder_labels_for_submission(results)
                #print (results)
                data_rows.append([audio_filename] + results)
    #             print([audio_filename] + results)
        writer.writerows(data_rows)
    
    # Clean up.
    del ckpt
    del model
    torch.cuda.empty_cache()

# Make Predictions

##### N.B: checkpoint must have been trained with same model architecture

## Make Coarse predictions

In [34]:
 make_results_csv(RESULT_CSV_COARSE_PATH, VGG_alt, COARSE_CHECKPOINT)

Making results file: csvs/results-20190715_141637-coarse.csv


## Make Fine predictions

In [77]:
for coarse_idx in range(NUM_COARSE_LABELS):
#     print(coarse_idx)
    label_start, label_end, num_fine_classes = get_label_range(coarse_idx)
    make_fine_results_csv(RESULT_CSV_FINE_PATH_TEMPLATE, VGG_alt_fine, ckpt_paths_fine[coarse_idx], 
                     use_fine_model=True, coarse_idx=coarse_idx, 
                     label_start=label_start, label_end=label_end, num_fine_classes=num_fine_classes)
    print("total_files ", total_files)

0       1_engine



total_files  1401
1       2_machinery-impact



total_files  1844
total_files  1844
3       4_powered-saw



total_files  2287
4       5_alert-signal



total_files  2730
5       6_music



total_files  3173
6       7_human-voice



total_files  3616
total_files  3616


## Combine separate prediction CSVs into final output CSV

1. Load coarse-model results CSV.
2. For each coarse class with a high score, copy in the fine-level results.

In [78]:
df = pd.read_csv(RESULT_CSV_COARSE_PATH)
print(RESULT_CSV_COARSE_PATH)
df.head()


csvs/results-20190715_141637-coarse.csv


,audio_filename,1-1_small-sounding-engine,1-2_medium-sounding-engine,1-3_large-sounding-engine,1-X_engine-of-uncertain-size,2-1_rock-drill,2-2_jackhammer,2-3_hoe-ram,2-4_pile-driver,2-X_other-unknown-impact-machinery,...,7-X_other-unknown-human-voice,8-1_dog-barking-whining,1_engine,2_machinery-impact,3_non-machinery-impact,4_powered-saw,5_alert-signal,6_music,7_human-voice,8_dog
0,00_001177.wav,0.038288,0.122052,0.502112,0.101482,1.838860e-03,8.821300e-04,6.156426e-04,2.825153e-05,0.032948,...,0.000232,0.000566,0.696243,0.058200,0.037064,0.007079,0.023692,0.000129,0.012542,0.000551
1,04_000468.wav,0.001285,0.004436,0.001538,0.001433,1.651226e-08,7.369137e-07,2.464860e-07,1.526348e-04,0.001204,...,0.014872,0.029644,0.012437,0.015715,0.003796,0.000192,0.005412,0.011788,0.922762,0.029466
2,04_000182.wav,0.017328,0.065575,0.099614,0.026444,1.078371e-05,6.772748e-05,3.545732e-04,4.953779e-03,0.026367,...,0.110802,0.010844,0.236392,0.075319,0.035246,0.001487,0.040328,0.028849,0.846842,0.010727
3,03_001573.wav,0.006351,0.042999,0.016009,0.044127,1.119302e-04,5.963831e-04,9.065376e-06,4.337600e-04,0.003769,...,0.008019,0.185584,0.082052,0.035761,0.035577,0.012118,0.456379,0.073907,0.181871,0.184687
4,04_000588.wav,0.015593,0.244084,0.639739,0.032929,9.380853e-04,1.698513e-03,2.567723e-04,2.031658e-09,0.001455,...,0.000043,0.005657,0.865977,0.020132,0.001470,0.001156,0.025869,0.000029,0.006563,0.005812


In [79]:
dfs_fine = defaultdict(lambda: None)
label_ranges = {}
for coarse_idx in range(NUM_COARSE_LABELS):
    csv_path = RESULT_CSV_FINE_PATH_TEMPLATE % coarse_idx
    label_start, label_end, _ = get_label_range(coarse_idx)
     # convert from starting fine labels at col 8 to starting at col 1
    label_start -= NUM_COARSE_LABELS - 1  # -1 is to account for audio_filename in col 0
    label_end -= NUM_COARSE_LABELS - 1
    label_ranges[coarse_idx] = (label_start, label_end)
    print(label_start, "  ", label_end)
    if os.path.exists(csv_path):
        dfs_fine[coarse_idx] = pd.read_csv(csv_path)

1    5
5    10
10    11
11    15
15    20
20    24
24    29
29    30


In [80]:
label_ranges

{0: (1, 5),
 1: (5, 10),
 2: (10, 11),
 3: (11, 15),
 4: (15, 20),
 5: (20, 24),
 6: (24, 29),
 7: (29, 30)}

In [81]:
#dfs_fine[1].head()

In [82]:
df.values.shape

(443, 38)

In [84]:
for i in tqdm_notebook(range(len(df))):
#     print("length ", len(df))
#     print("df_fine " , df_fine.values.shape)
    # Find the coarse-predicitons that are over-threshold OR are max over all coarse categories.
    # Find argmax.
    coarse_preds = df.values[i][1 + NUM_FINE_LABELS:]
#     print(coarse_preds)
    best_coarse_idx = np.argmax(coarse_preds)
    for coarse_idx in range(NUM_COARSE_LABELS):
        start_idx, end_idx = label_ranges[coarse_idx]
        coarse_prob = df.values[i][1 + NUM_FINE_LABELS + coarse_idx]
#         print(coarse_preds)
        if coarse_prob > COARSE_PREDICT_THRESHOLD or coarse_idx == best_coarse_idx:  # + 1 is because of filename in col 0
            df_fine = dfs_fine[coarse_idx]
#             print(type(df_fine))
#             print(df_fine.shape)
#             print(df_fine)
            if df_fine is None:
                assert end_idx - start_idx == 1
                df.iloc[i, start_idx] = coarse_prob
            else:
                # Copy fine-predictions over into the results data.
#                 print(start_idx, "   ",  end_idx)
#                 print(i)
#                 print("coarse_idx ", coarse_idx)
#                 print(start_idx, "   ", end_idx)
#                 print("df_fine " , df_fine.values.shape)
#                 print("df_fine " , df_fine.values.shape)
#                 print(df_fine.values[i][0])
#                 print(len(df_fine.values[i]))  
#                 print(df_fine.values[i][start_idx:end_idx])
#                 print()
                df.iloc[i, start_idx:end_idx] = df_fine.values[i][start_idx:end_idx]
                

In [85]:
df.head()

,audio_filename,1-1_small-sounding-engine,1-2_medium-sounding-engine,1-3_large-sounding-engine,1-X_engine-of-uncertain-size,2-1_rock-drill,2-2_jackhammer,2-3_hoe-ram,2-4_pile-driver,2-X_other-unknown-impact-machinery,...,7-X_other-unknown-human-voice,8-1_dog-barking-whining,1_engine,2_machinery-impact,3_non-machinery-impact,4_powered-saw,5_alert-signal,6_music,7_human-voice,8_dog
0,00_001177.wav,0.019638,0.083289,0.498525,0.027672,1.838860e-03,8.821300e-04,6.156426e-04,2.825153e-05,0.032948,...,0.000232,0.000566,0.696243,0.058200,0.037064,0.007079,0.023692,0.000129,0.012542,0.000551
1,04_000468.wav,0.001285,0.004436,0.001538,0.001433,1.651226e-08,7.369137e-07,2.464860e-07,1.526348e-04,0.001204,...,0.005020,0.029644,0.012437,0.015715,0.003796,0.000192,0.005412,0.011788,0.922762,0.029466
2,04_000182.wav,0.017328,0.065575,0.099614,0.026444,1.078371e-05,6.772748e-05,3.545732e-04,4.953779e-03,0.026367,...,0.005362,0.010844,0.236392,0.075319,0.035246,0.001487,0.040328,0.028849,0.846842,0.010727
3,03_001573.wav,0.006351,0.042999,0.016009,0.044127,1.119302e-04,5.963831e-04,9.065376e-06,4.337600e-04,0.003769,...,0.008019,0.185584,0.082052,0.035761,0.035577,0.012118,0.456379,0.073907,0.181871,0.184687
4,04_000588.wav,0.016314,0.157480,0.415551,0.015849,9.380853e-04,1.698513e-03,2.567723e-04,2.031658e-09,0.001455,...,0.000043,0.005657,0.865977,0.020132,0.001470,0.001156,0.025869,0.000029,0.006563,0.005812


In [45]:
df.to_csv(RESULT_CSV_FINAL_PATH)

In [46]:
RESULT_CSV_FINAL_PATH

'csvs/results-20190715_141637-final.csv'

# Eval

In [54]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  This is the result from /dcase/output/models_Jianyu/one-part-1-models-logmel-best6-originalmodel_rmsprop-learningratechangebiggerinCallback
##  train_branches_20190713_1_one_part_one_model_OriginalModel_LeanringRateChangeMuchlower.py
if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190713_112132-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml


sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6254867629685286
 * Micro F1-score (@0.5): 0.4909819639278557
 * Macro AUPRC:           0.3857762522385675
 * Coarse Tag AUPRC:
      - 1: 0.6084437663572311
      - 2: 0.14288025423208833
      - 3: 0.3336595526102611
      - 4: 0.33066483112050044
      - 5: 0.668508235927582
      - 6: 0.1563059224638172
      - 7: 0.8457474551970598
      - 8: 0.0
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() i

In [28]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  This is the result from /dcase/output/models_Jianyu/one-part-1-models-logmel-best6-originalmodel_rmsprop-learningratechangebiggerinCallback
##  train_branches_20190713_1_one_part_one_model_OriginalModel_LeanringRateChangeMuchlower.py
if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190713_103400-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml


sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6352275033460464
 * Micro F1-score (@0.5): 0.473953013278856
 * Macro AUPRC:           0.3859955932100825
 * Coarse Tag AUPRC:
      - 1: 0.6421604848641354
      - 2: 0.14290489694838895
      - 3: 0.3336595526102611
      - 4: 0.28717011575218754
      - 5: 0.6714119230258853
      - 6: 0.1563059224638172
      - 7: 0.8543518500159846
      - 8: 0.0
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() i

In [24]:
#################################  this result is add to the report 



# run a command similar to this on the command line to get results  # 20190609_031224

##  This is the result from /dcase/output/models_Jianyu/one-part-1-models-logmel-best5-rmsprop-learningratechangebiggerinCallback
##  train_branches_20190711_3_one_part_one_model_LeanringRateChangeSlower.py

if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190715_141637-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml


sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6429200022744848
 * Micro F1-score (@0.5): 0.5163853028798411
 * Macro AUPRC:           0.41221486377413197
 * Coarse Tag AUPRC:
      - 1: 0.6866141325176544
      - 2: 0.12112664107420393
      - 3: 0.4020905123120714
      - 4: 0.3870551691374258
      - 5: 0.659413704156418
      - 6: 0.251310185085831
      - 7: 0.7847798844483058
      - 8: 0.005328681461145428
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_num

In [47]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  This is the result from /dcase/output/models_Jianyu/one-part-1-models-logmel-best4-rmsprop-learningratechangebiggerinCallback
##  train_branches_20190711_3_one_part_one_model_LeanringRateChangeSlower.py
if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190715_141637-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml

sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6496101347404405
 * Micro F1-score (@0.5): 0.5005045408678103
 * Macro AUPRC:           0.4106011898721246
 * Coarse Tag AUPRC:
      - 1: 0.6689075318877663
      - 2: 0.13501270495855258
      - 3: 0.4020905123120714
      - 4: 0.33732866485418367
      - 5: 0.6505729200529182
      - 6: 0.251310185085831
      - 7: 0.834258318364528
      - 8: 0.005328681461145428
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_num

In [78]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  This is the result from one-part-1-models-logmel-best2-rmsp -dropoutlow
##  train_branches_20190711_2_one_part_one_model_no_dropout.py
if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190712_133112-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml

sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6363211133162249
 * Micro F1-score (@0.5): 0.4710743801652893
 * Macro AUPRC:           0.38647299543112723
 * Coarse Tag AUPRC:
      - 1: 0.654099839967299
      - 2: 0.1471312850330832
      - 3: 0.3336595526102611
      - 4: 0.3052075983279423
      - 5: 0.6632979242445739
      - 6: 0.1563059224638172
      - 7: 0.8320818408020411
      - 8: 0.0
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() in

In [51]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  This is the result from one-part-1-models-logmel-best2-rmsp 
##  train_branches_20190711_2_one_part_one_model.py     optimizer = rmsprop 
if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190712_113151-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml

sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6231496003383689
 * Micro F1-score (@0.5): 0.5010060362173039
 * Macro AUPRC:           0.38623738657528395
 * Coarse Tag AUPRC:
      - 1: 0.6343790243886206
      - 2: 0.16253583808623515
      - 3: 0.3336595526102611
      - 4: 0.32327465520753845
      - 5: 0.6664129472290485
      - 6: 0.1563059224638172
      - 7: 0.8133311526167504
      - 8: 0.0
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero()

In [27]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  This is the result from one-part-1-models-logmel-best 
##  train_branches_20190711_2_one_part_one_model.py     optimizer = adam  
if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190712_095434-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml

sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6361358901902983
 * Micro F1-score (@0.5): 0.4969818913480885
 * Macro AUPRC:           0.39259460285229086
 * Coarse Tag AUPRC:
      - 1: 0.6196898192139044
      - 2: 0.1658941086630264
      - 3: 0.3336595526102611
      - 4: 0.34424915961788777
      - 5: 0.6686388310242353
      - 6: 0.1563059224638172
      - 7: 0.8523194292251949
      - 8: 0.0
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() 

In [25]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  not sure which result it comes from, maybe the /dcase/output/model/best/   since it is also June 10
if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190610_231917-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml

sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6560867707337048
 * Micro F1-score (@0.5): 0.3982202447163515
 * Macro AUPRC:           0.40059566061982355
 * Coarse Tag AUPRC:
      - 1: 0.6405304300056462
      - 2: 0.21380721739093014
      - 3: 0.3336595526102611
      - 4: 0.2761679795550256
      - 5: 0.70229675965
      - 6: 0.1563059224638172
      - 7: 0.8819974232829082
      - 8: 0.0
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() inste

In [ ]:
# python extract_embedding.py "D:\DCASE_2019\annotations.csv" $SONYC_UST_PATH/data $SONYC_UST_PATH/features $SONYC_UST_PATH/vggish

In [26]:
# run a command similar to this on the command line to get results  # 20190609_031224

##  this is actuall  results  from  /dcase/trained-models-branches2/20190607.......


if not LINUX:
    !python evaluate_predictions.py results_2019-5-6-aug.csv "D:\DCASE_2019\annotations.csv" "D:\DCASE_2019\dcase-ust-taxonomy.yaml"
else:
    !python evaluate_predictions.py csvs/results-20190712_084152-final.csv /dcase/datasets-dcase-2019-5/annotations.csv /dcase/datasets-dcase-2019-5/dcase-ust-taxonomy.yaml

sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  thresh_0pt5_idx = (eval_df['threshold'] >= 0.5).nonzero()[0][0]
Fine level evaluation:
 * Micro AUPRC:           0.6293865759512395
 * Micro F1-score (@0.5): 0.49899799599198397
 * Macro AUPRC:           0.39545724490363043
 * Coarse Tag AUPRC:
      - 1: 0.6044598965450919
      - 2: 0.20886913422339365
      - 3: 0.3336595526102611
      - 4: 0.3436435915466351
      - 5: 0.6959986741419814
      - 6: 0.1563059224638172
      - 7: 0.8207211876978632
      - 8: 0.0
sys:1: DtypeWarning: Columns (40,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
evaluate_predictions.py:36: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero()